# 主要内容
好习惯，数据文件夹分为input和output

注input文件里放下面这个链接的比赛下载的数据分别为`train_set.csv`和`test_set.csv`

[「二分类算法」提供银行精准营销解决方案 ](https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3) 案例
- 特征工程前的准备 (拼接测试训练集)
- 数据探索Exploratory Data Analysis，简称EDA
- 字符型特征转数字(LabelEncoder)
- lightgbm baseline模型
- kfold

[github源代码](https://github.com/PandasCute/Provide-Banks-with-precision-marketing-solutions-Provide-Banks-with-precision-marketing-solutions)


[提交结果](https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3/submit)

In [1]:
#读取csv格式文件
import pandas as pd
pd.set_option('display.max_columns', None)
path = './input/' #用户的路径环境
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [2]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [3]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb

In [4]:
LABEL = 'y'
test[LABEL] = -1
cat_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

UNUSE = ['ID', LABEL]
# cat_features = [ft for ft in train.select_dtypes(object).columns if ft not in UNUSE]
# num_features = [ft for ft in train.select_dtypes('number').columns if ft not in UNUSE]

features = cat_features + num_features
data = pd.concat([train,test])

In [5]:
# def feat_count(df, df_feature,fe,value,name=""):
    
#     df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
#     if not name:
#         df_count.columns = fe + [value+"_%s_count" % ("_".join(fe[0]))]
#     else:
#         df_count.columns = fe + [name]
#     df = df.merge(df_count, on=fe, how="left")#.fillna(0)
#     return df,df_count.columns[1]

# 这一步的操作令人迷惑
# for df in features:
#     data = feat_count(data, data, ['ID'], df)[0]

In [6]:
# 计数特征
for ft in cat_features:
    temp = data.groupby(ft).size().reset_index().rename(columns={0: 'count_'+ft})
    data = data.merge(temp, 'left', on=ft)

In [7]:
# 测试用例
# data.groupby(['day','month','job']).size().reset_index().rename(columns={0: 'job'})
def feature_count(data, features):
    # 任选2个以上的特征进行组合统计其个数
    feature_name = 'count' + '_' + '_'.join(features)
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name

new_features1=[]
for f in cat_features:
    if f not in ['day','month']:
        data,_=feature_count(data,['day','month',f])
        new_features1.append(_)

In [8]:
# 类别型特征与其他特征组合
def get_new_columns(name, aggs):
    # name 用于和其他特征组合特初始特征名
    # aggs 其他特征名
    new_columns = []
    for k in aggs.keys():
        for agg in aggs[k]:
            # agg 每种特征的具体操作
            if str(type(agg))=="<class 'function'>":
                new_columns.append(name + '_' + k + '_' + 'other')
            else:
                new_columns.append(name + '_' + k + '_' + agg)
    return new_columns
    
for ft in tqdm(cat_features):
    
    aggs={}
    for s in cat_features:
        aggs[s]=['nunique']
    for den in num_features:
        aggs[den]=['mean','max','min','std']
    aggs.pop(ft)  # 不需要与自己组合
    
    temp=data.groupby(ft).agg(aggs).reset_index()  # reset_index 否则会以 ft 为索引列
    temp.columns=[ft]+get_new_columns(ft, aggs)  # 重命名columns以便做merge操作
    data=pd.merge(data,temp,on=ft,how='left')

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:15<00:00,  2.70s/it]


In [9]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold

# cat_features 不变
data['new_con'] = data['job'].astype(str)  # data['new_con'] == data['job']

for ft in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + ' ' + data[ft].astype(str)
    
# 类别型特征one-hot
data = pd.concat([data, pd.get_dummies(data[cat_features])], axis=1)
# 类别转数字
data[cat_features] = data[cat_features].apply(lambda x: LabelEncoder().fit_transform(data[x.name]))        
vector_feature = 'new_con'


res = data[data.y==-1][['ID']]  # 结果
# 向量化特征
vector_data = CountVectorizer().fit_transform(data[vector_feature]).toarray()
data = pd.concat([data, pd.DataFrame(vector_data)], axis=1)

num_features = [ft for ft in data.select_dtypes('number').columns if ft not in UNUSE]

train_x = data[data.y!=-1][num_features]
train_y = data[data.y!=-1][LABEL]
test_x = data[data.y==-1][num_features]

In [13]:
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

def get_clf(random_state):
    clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
        max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
        subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.02, random_state=random_state)
    clf.fit(x_train, y_train, 
            eval_set=[(x_train, y_train),(x_val, y_val)],
            eval_metric='auc',
            early_stopping_rounds=100)
    return clf
    
res['pred'] = 0
random_state = 2017

for train_idx, val_idx in kfold.split(train_x):
    x_train = train_x.loc[train_idx]  # 若报错，去掉.loc
    y_train = train_y.loc[train_idx]
    x_val = train_x.loc[val_idx]
    y_val = train_y.loc[val_idx]
    
    clf = get_clf(random_state)
    random_state += 1
    res['pred'] += clf.predict_proba(test_x)[:, 1]

res['pred'] = res['pred']/5
res.to_csv('./my_basesubmission2.csv', index=False)

[1]	training's auc: 0.91881	valid_1's auc: 0.902993
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.924408	valid_1's auc: 0.905155
[3]	training's auc: 0.925264	valid_1's auc: 0.906243
[4]	training's auc: 0.929097	valid_1's auc: 0.912923
[5]	training's auc: 0.933549	valid_1's auc: 0.920143
[6]	training's auc: 0.934102	valid_1's auc: 0.920967
[7]	training's auc: 0.935236	valid_1's auc: 0.92281
[8]	training's auc: 0.936017	valid_1's auc: 0.9234
[9]	training's auc: 0.936629	valid_1's auc: 0.924041
[10]	training's auc: 0.936842	valid_1's auc: 0.923932
[11]	training's auc: 0.93807	valid_1's auc: 0.926339
[12]	training's auc: 0.939141	valid_1's auc: 0.927935
[13]	training's auc: 0.940449	valid_1's auc: 0.92902
[14]	training's auc: 0.940688	valid_1's auc: 0.928977
[15]	training's auc: 0.94094	valid_1's auc: 0.92899
[16]	training's auc: 0.942144	valid_1's auc: 0.93038
[17]	training's auc: 0.942922	valid_1's auc: 0.931853
[18]	training's auc: 0.943182	valid_1

[151]	training's auc: 0.959197	valid_1's auc: 0.938062
[152]	training's auc: 0.959284	valid_1's auc: 0.938096
[153]	training's auc: 0.959394	valid_1's auc: 0.938148
[154]	training's auc: 0.959534	valid_1's auc: 0.938147
[155]	training's auc: 0.959636	valid_1's auc: 0.938161
[156]	training's auc: 0.959731	valid_1's auc: 0.938147
[157]	training's auc: 0.959817	valid_1's auc: 0.938131
[158]	training's auc: 0.95993	valid_1's auc: 0.938183
[159]	training's auc: 0.960062	valid_1's auc: 0.93819
[160]	training's auc: 0.960154	valid_1's auc: 0.938209
[161]	training's auc: 0.960218	valid_1's auc: 0.93817
[162]	training's auc: 0.960301	valid_1's auc: 0.938253
[163]	training's auc: 0.960394	valid_1's auc: 0.938257
[164]	training's auc: 0.960492	valid_1's auc: 0.938248
[165]	training's auc: 0.960602	valid_1's auc: 0.938345
[166]	training's auc: 0.960674	valid_1's auc: 0.938391
[167]	training's auc: 0.960791	valid_1's auc: 0.938389
[168]	training's auc: 0.960892	valid_1's auc: 0.938414
[169]	trainin

[300]	training's auc: 0.971481	valid_1's auc: 0.93965
[301]	training's auc: 0.971563	valid_1's auc: 0.939652
[302]	training's auc: 0.971651	valid_1's auc: 0.939646
[303]	training's auc: 0.97172	valid_1's auc: 0.939638
[304]	training's auc: 0.971771	valid_1's auc: 0.939648
[305]	training's auc: 0.971851	valid_1's auc: 0.93965
[306]	training's auc: 0.97191	valid_1's auc: 0.939689
[307]	training's auc: 0.972008	valid_1's auc: 0.939686
[308]	training's auc: 0.97209	valid_1's auc: 0.939732
[309]	training's auc: 0.972133	valid_1's auc: 0.93972
[310]	training's auc: 0.97219	valid_1's auc: 0.939756
[311]	training's auc: 0.972255	valid_1's auc: 0.939706
[312]	training's auc: 0.972342	valid_1's auc: 0.93972
[313]	training's auc: 0.972396	valid_1's auc: 0.9397
[314]	training's auc: 0.972483	valid_1's auc: 0.939677
[315]	training's auc: 0.972574	valid_1's auc: 0.93966
[316]	training's auc: 0.972643	valid_1's auc: 0.939634
[317]	training's auc: 0.972758	valid_1's auc: 0.9397
[318]	training's auc: 0

[37]	training's auc: 0.948038	valid_1's auc: 0.927851
[38]	training's auc: 0.948119	valid_1's auc: 0.927803
[39]	training's auc: 0.948301	valid_1's auc: 0.928005
[40]	training's auc: 0.948528	valid_1's auc: 0.928244
[41]	training's auc: 0.948628	valid_1's auc: 0.928256
[42]	training's auc: 0.948825	valid_1's auc: 0.928283
[43]	training's auc: 0.948946	valid_1's auc: 0.928386
[44]	training's auc: 0.949102	valid_1's auc: 0.928504
[45]	training's auc: 0.949274	valid_1's auc: 0.928454
[46]	training's auc: 0.949396	valid_1's auc: 0.928451
[47]	training's auc: 0.949545	valid_1's auc: 0.928569
[48]	training's auc: 0.949624	valid_1's auc: 0.928658
[49]	training's auc: 0.949859	valid_1's auc: 0.929009
[50]	training's auc: 0.949948	valid_1's auc: 0.929093
[51]	training's auc: 0.950019	valid_1's auc: 0.929183
[52]	training's auc: 0.95011	valid_1's auc: 0.929184
[53]	training's auc: 0.950216	valid_1's auc: 0.92918
[54]	training's auc: 0.950375	valid_1's auc: 0.929375
[55]	training's auc: 0.950564	

[187]	training's auc: 0.963258	valid_1's auc: 0.933809
[188]	training's auc: 0.963346	valid_1's auc: 0.933832
[189]	training's auc: 0.963422	valid_1's auc: 0.933887
[190]	training's auc: 0.963506	valid_1's auc: 0.933848
[191]	training's auc: 0.963591	valid_1's auc: 0.933822
[192]	training's auc: 0.963695	valid_1's auc: 0.933864
[193]	training's auc: 0.963794	valid_1's auc: 0.933881
[194]	training's auc: 0.963877	valid_1's auc: 0.93394
[195]	training's auc: 0.96396	valid_1's auc: 0.933923
[196]	training's auc: 0.964026	valid_1's auc: 0.933947
[197]	training's auc: 0.964121	valid_1's auc: 0.933963
[198]	training's auc: 0.964211	valid_1's auc: 0.933955
[199]	training's auc: 0.964295	valid_1's auc: 0.934001
[200]	training's auc: 0.964385	valid_1's auc: 0.934025
[201]	training's auc: 0.964451	valid_1's auc: 0.934077
[202]	training's auc: 0.964555	valid_1's auc: 0.934102
[203]	training's auc: 0.964689	valid_1's auc: 0.934124
[204]	training's auc: 0.964781	valid_1's auc: 0.934166
[205]	traini

...

[156]	training's auc: 0.959293	valid_1's auc: 0.941894
[157]	training's auc: 0.959392	valid_1's auc: 0.941933
[158]	training's auc: 0.959491	valid_1's auc: 0.941913
[159]	training's auc: 0.959573	valid_1's auc: 0.941979
[160]	training's auc: 0.959744	valid_1's auc: 0.94198
[161]	training's auc: 0.959837	valid_1's auc: 0.942034
[162]	training's auc: 0.959911	valid_1's auc: 0.942072
[163]	training's auc: 0.959984	valid_1's auc: 0.942111
[164]	training's auc: 0.960101	valid_1's auc: 0.942152
[165]	training's auc: 0.96021	valid_1's auc: 0.942186
[166]	training's auc: 0.960275	valid_1's auc: 0.942219
[167]	training's auc: 0.960365	valid_1's auc: 0.942233
[168]	training's auc: 0.960437	valid_1's auc: 0.942249
[169]	training's auc: 0.960522	valid_1's auc: 0.942267
[170]	training's auc: 0.960587	valid_1's auc: 0.942267
[171]	training's auc: 0.960698	valid_1's auc: 0.942328
[172]	training's auc: 0.960802	valid_1's auc: 0.942338
[173]	training's auc: 0.960882	valid_1's auc: 0.942377
[174]	traini

[305]	training's auc: 0.97157	valid_1's auc: 0.94373
[306]	training's auc: 0.971644	valid_1's auc: 0.943734
[307]	training's auc: 0.971712	valid_1's auc: 0.943742
[308]	training's auc: 0.971804	valid_1's auc: 0.943728
[309]	training's auc: 0.971859	valid_1's auc: 0.943754
[310]	training's auc: 0.971915	valid_1's auc: 0.943764
[311]	training's auc: 0.971986	valid_1's auc: 0.943776
[312]	training's auc: 0.972041	valid_1's auc: 0.943787
[313]	training's auc: 0.972094	valid_1's auc: 0.943766
[314]	training's auc: 0.972159	valid_1's auc: 0.943766
[315]	training's auc: 0.972213	valid_1's auc: 0.94378
[316]	training's auc: 0.972283	valid_1's auc: 0.943738
[317]	training's auc: 0.972343	valid_1's auc: 0.943769
[318]	training's auc: 0.972394	valid_1's auc: 0.943763
[319]	training's auc: 0.972468	valid_1's auc: 0.943724
[320]	training's auc: 0.972547	valid_1's auc: 0.94375
[321]	training's auc: 0.972625	valid_1's auc: 0.94378
[322]	training's auc: 0.972677	valid_1's auc: 0.943792
[323]	training'

[454]	training's auc: 0.980223	valid_1's auc: 0.943601
[455]	training's auc: 0.980271	valid_1's auc: 0.943603
[456]	training's auc: 0.980316	valid_1's auc: 0.943609
[457]	training's auc: 0.980362	valid_1's auc: 0.943631
[458]	training's auc: 0.980404	valid_1's auc: 0.943672
[459]	training's auc: 0.980437	valid_1's auc: 0.943664
[460]	training's auc: 0.980473	valid_1's auc: 0.94365
[461]	training's auc: 0.980525	valid_1's auc: 0.943621
[462]	training's auc: 0.980573	valid_1's auc: 0.943604
[463]	training's auc: 0.980628	valid_1's auc: 0.943598
[464]	training's auc: 0.980685	valid_1's auc: 0.9436
[465]	training's auc: 0.980725	valid_1's auc: 0.943614
[466]	training's auc: 0.980771	valid_1's auc: 0.94362
[467]	training's auc: 0.980818	valid_1's auc: 0.943637
[468]	training's auc: 0.980881	valid_1's auc: 0.943632
[469]	training's auc: 0.980927	valid_1's auc: 0.94363
[470]	training's auc: 0.980986	valid_1's auc: 0.943606
[471]	training's auc: 0.981045	valid_1's auc: 0.943593
[472]	training'

[122]	training's auc: 0.957644	valid_1's auc: 0.93434
[123]	training's auc: 0.95775	valid_1's auc: 0.934452
[124]	training's auc: 0.957855	valid_1's auc: 0.934469
[125]	training's auc: 0.957976	valid_1's auc: 0.934442
[126]	training's auc: 0.958084	valid_1's auc: 0.93457
[127]	training's auc: 0.958238	valid_1's auc: 0.934577
[128]	training's auc: 0.958329	valid_1's auc: 0.934669
[129]	training's auc: 0.958397	valid_1's auc: 0.934702
[130]	training's auc: 0.95853	valid_1's auc: 0.934743
[131]	training's auc: 0.958626	valid_1's auc: 0.934799
[132]	training's auc: 0.958746	valid_1's auc: 0.934838
[133]	training's auc: 0.958853	valid_1's auc: 0.93488
[134]	training's auc: 0.958941	valid_1's auc: 0.934939
[135]	training's auc: 0.959095	valid_1's auc: 0.934963
[136]	training's auc: 0.959221	valid_1's auc: 0.935017
[137]	training's auc: 0.959351	valid_1's auc: 0.935052
[138]	training's auc: 0.959446	valid_1's auc: 0.935065
[139]	training's auc: 0.959547	valid_1's auc: 0.935048
[140]	training'

[271]	training's auc: 0.97088	valid_1's auc: 0.936189
[272]	training's auc: 0.970963	valid_1's auc: 0.936179
[273]	training's auc: 0.97103	valid_1's auc: 0.936177
[274]	training's auc: 0.971116	valid_1's auc: 0.9362
[275]	training's auc: 0.971197	valid_1's auc: 0.936191
[276]	training's auc: 0.97127	valid_1's auc: 0.936203
[277]	training's auc: 0.971333	valid_1's auc: 0.936183
[278]	training's auc: 0.971405	valid_1's auc: 0.936208
[279]	training's auc: 0.971476	valid_1's auc: 0.936195
[280]	training's auc: 0.971562	valid_1's auc: 0.936196
[281]	training's auc: 0.97164	valid_1's auc: 0.936261
[282]	training's auc: 0.971722	valid_1's auc: 0.936241
[283]	training's auc: 0.971805	valid_1's auc: 0.936223
[284]	training's auc: 0.971857	valid_1's auc: 0.936206
[285]	training's auc: 0.971924	valid_1's auc: 0.936196
[286]	training's auc: 0.971967	valid_1's auc: 0.936241
[287]	training's auc: 0.972022	valid_1's auc: 0.936242
[288]	training's auc: 0.97209	valid_1's auc: 0.936265
[289]	training's 

[提交结果](https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3/submit)